<a href="https://colab.research.google.com/github/RanxduG/Translation_Transformer/blob/main/Colab_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Install required packages testing

In [1]:
!pip install torch torchvision torchaudio
!pip install transformers datasets
!pip install matplotlib seaborn
!pip install sentencepiece

##Importing libraries

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import pickle
import os
from tqdm import tqdm
from datasets import load_dataset
import random

##Initializing Datasets

In [3]:
# Upload your transformer_sinhala.py file to Colab and import it
from transformer import Transformer, get_device

# Set device
device = get_device()
print(f"Using device: {device}")

# Load dataset from HuggingFace
print("Loading FLORES-200 dataset...")
try:
    # Load FLORES-200 dataset
    dataset = load_dataset("facebook/flores", "eng_Latn-sin_Sinh")

    # Extract English and Sinhala sentences
    english_sentences = []
    sinhala_sentences = []

    # Use dev split for smaller dataset (good for Colab)
    data_split = dataset['dev']

    for example in data_split:
        english_sentences.append(example['sentence_eng_Latn'])
        sinhala_sentences.append(example['sentence_sin_Sinh'])

    print(f"Loaded {len(english_sentences)} sentence pairs from FLORES-200")

except Exception as e:
    print(f"Error loading FLORES-200: {e}")
    print("Falling back to alternative dataset...")

    # Alternative: Load sentence alignment dataset
    try:
        dataset = load_dataset("NLPC-UOM/sentence_alignment_dataset-Sinhala-Tamil-English")

        # Extract English and Sinhala sentences
        english_sentences = []
        sinhala_sentences = []

        # Use train split and limit to first 1000 examples
        data_split = dataset['train']

        for i, example in enumerate(data_split):
            if i >= 1000:  # Limit dataset size for Colab
                break
            if 'english' in example and 'sinhala' in example:
                if example['english'] and example['sinhala']:
                    english_sentences.append(example['english'].strip())
                    sinhala_sentences.append(example['sinhala'].strip())

        print(f"Loaded {len(english_sentences)} sentence pairs from NLPC-UOM dataset")

    except Exception as e2:
        print(f"Error loading alternative dataset: {e2}")
        print("Using sample dataset with real Sinhala script...")

        # Fallback to sample dataset with real Sinhala script
        # Sample dataset with cleaned data
        english_sentences = [
            "Hello",
            "How are you?",
            "Good morning",
            "Good evening",
            "Thank you",
            "You are welcome",
            "I am fine",
            "What is your name?",
            "Nice to meet you",
            "How old are you?",
            "Where are you from?",
            "I love you",
            "Goodbye",
            "See you later",
            "Have a nice day",
            "I am hungry",
            "I am tired",
            "What time is it?",
            "It is raining",
            "The weather is nice",
            "I want to eat",
            "I want to sleep",
            "I am happy",
            "I am sad",
            "Help me",
            "I do not understand",
            "Speak slowly",
            "Can you help me?",
            "Where is the bathroom?",
            "How much does it cost?",
            "I am learning Sinhala",
            "The book is on the table",
            "I like this food",
            "The car is red",
            "She is beautiful",
            "He is tall",
            "We are friends",
            "They are students",
            "I go to school",
            "I work in an office"
        ]

        sinhala_sentences = [
            "ආයුබෝවන්",
            "ඔබ කෙසේද?",
            "සුභ උදෑසනක්",
            "සුභ සන්ධ්‍යාවක්",
            "ස්තූතියි",
            "ආයුබෝවන්",
            "මම හොඳයි",
            "ඔබගේ නම මොකක්ද?",
            "ඔබව හමුවීමට සතුටුයි",
            "ඔබට වයස කීයද?",
            "ඔබ කොහෙන්ද?",
            "මම ඔබට ආදරෙයි",
            "ආයුබෝවන්",
            "ඉදිරියෙදී හමුවෙමු",
            "හොඳ දවසක් ගත කරන්න",
            "මට කිඩෙන් හිතෙනවා",
            "මට කඩා වැටෙනවා",
            "වේලාව කීයද?",
            "වැස්ස එනවා",
            "කාලගුණය හොඳයි",
            "මට කන්න ඕන",
            "මට නිදාගන්න ඕන",
            "මට සතුටුයි",
            "මට දුකයි",
            "මට උදව් කරන්න",
            "මට තේරෙන්නේ නෑ",
            "සෙමින් කතා කරන්න",
            "ඔබට මට උදව් කරන්න පුළුවන්ද?",
            "නාන කාමරය කොහෙද?",
            "මේක කීයද?",
            "මම සිංහල ඉගෙන ගන්නවා",
            "පොත මේසය උඩින් තියෙනවා",
            "මට මේ කෑම කමතියි",
            "කාර් එක රතුයි",
            "ඇය ලස්සනයි",
            "ඔහු උසයි",
            "අපි යාළුවෝයි",
            "ඔවුන් සිසුන්",
            "මම පාසලට යනවා",
            "මම කාර්යාලයක වැඩ කරනවා"
        ]

# Display sample data
print("\nSample data:")
for i in range(5):
    print(f"English: {english_sentences[i]}")
    print(f"Sinhala: {sinhala_sentences[i]}")
    print("---")

Using device: cuda
Loading FLORES-200 dataset...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Error loading FLORES-200: Loading a dataset cached in a LocalFileSystem is not supported.
Falling back to alternative dataset...


Error loading alternative dataset: Invalid pattern: '**' can only be an entire path component
Using sample dataset with real Sinhala script...

Sample data:
English: Hello
Sinhala: ආයුබෝවන්
---
English: How are you?
Sinhala: ඔබ කෙසේද?
---
English: Good morning
Sinhala: සුභ උදෑසනක්
---
English: Good evening
Sinhala: සුභ සන්ධ්‍යාවක්
---
English: Thank you
Sinhala: ස්තූතියි
---


In [4]:
# Dataset class for parallel text
class TranslationDataset(Dataset):
    def __init__(self, english_sentences, sinhala_sentences):
        self.english_sentences = english_sentences
        self.sinhala_sentences = sinhala_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.sinhala_sentences[idx]

# Improved vocabulary creation with better character handling
def create_vocabulary(sentences, min_freq=1):
    # Character-level tokenization with cleaning
    char_counter = Counter()

    for sentence in sentences:
        # Clean the sentence and count characters
        cleaned_sentence = sentence.strip()
        for char in cleaned_sentence:
            char_counter[char] += 1

    # Create vocabulary with special tokens first
    vocab = ['<PAD>', '<START>', '<END>', '<UNK>']

    # Add characters sorted by frequency (most common first)
    for char, freq in char_counter.most_common():
        if freq >= min_freq and char not in vocab:
            vocab.append(char)

    # Create index mappings
    char_to_index = {char: i for i, char in enumerate(vocab)}
    index_to_char = {i: char for i, char in enumerate(vocab)}

    return char_to_index, index_to_char, vocab

# Create vocabularies
print("Creating vocabularies...")
english_to_index, index_to_english, english_vocab = create_vocabulary(english_sentences)
sinhala_to_index, index_to_sinhala, sinhala_vocab = create_vocabulary(sinhala_sentences)

print(f"English vocab size: {len(english_vocab)}")
print(f"Sinhala vocab size: {len(sinhala_vocab)}")
print(f"Sample English vocab: {english_vocab[:20]}")
print(f"Sample Sinhala vocab: {sinhala_vocab[:20]}")

Creating vocabularies...
English vocab size: 37
Sinhala vocab size: 49
Sample English vocab: ['<PAD>', '<START>', '<END>', '<UNK>', ' ', 'e', 'o', 'a', 'i', 't', 'n', 'r', 'h', 's', 'l', 'u', 'm', 'y', 'd', 'I']
Sample Sinhala vocab: ['<PAD>', '<START>', '<END>', '<UNK>', ' ', 'න', 'ම', '්', 'ය', 'ක', 'ව', 'ි', 'ු', 'ස', 'ා', 'ට', 'ද', 'ෙ', 'ත', 'ර']


##Initializing Model

In [5]:
# Define special tokens
START_TOKEN = '<START>'
END_TOKEN = '<END>'
PADDING_TOKEN = '<PAD>'
UNK_TOKEN = '<UNK>'

# Model hyperparameters (optimized for small dataset)
d_model = 128
ffn_hidden = 256
num_heads = 4
drop_prob = 0.1
num_layers = 3
max_sequence_length = 64
learning_rate = 0.001
batch_size = 8
num_epochs = 10

model = Transformer(
    d_model=d_model,
    ffn_hidden=ffn_hidden,
    num_heads=num_heads,
    drop_prob=drop_prob,
    num_layers=num_layers,
    max_sequence_length=max_sequence_length,
    si_vocab_size=len(sinhala_vocab),
    english_to_index=english_to_index,
    sinhala_to_index=sinhala_to_index,
    START_TOKEN=START_TOKEN,
    END_TOKEN=END_TOKEN,
    PADDING_TOKEN=PADDING_TOKEN
).to(device)

# Print model parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total model parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

# Create train/validation split
train_size = int(0.8 * len(english_sentences))
val_size = len(english_sentences) - train_size

# Shuffle data before splitting
combined = list(zip(english_sentences, sinhala_sentences))
random.shuffle(combined)
english_sentences, sinhala_sentences = zip(*combined)

train_english = list(english_sentences[:train_size])
train_sinhala = list(sinhala_sentences[:train_size])
val_english = list(english_sentences[train_size:])
val_sinhala = list(sinhala_sentences[train_size:])

print(f"Train samples: {len(train_english)}")
print(f"Validation samples: {len(val_english)}")

# Create datasets and dataloaders
train_dataset = TranslationDataset(train_english, train_sinhala)
val_dataset = TranslationDataset(val_english, val_sinhala)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=sinhala_to_index[PADDING_TOKEN])
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, verbose=True)

Total model parameters: 1,011,121
Trainable parameters: 1,011,121
Train samples: 32
Validation samples: 8


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [6]:
# Helper function to create attention masks
def create_masks(src_len, tgt_len, device):
    # Create decoder self-attention mask (causal mask)
    decoder_mask = torch.triu(torch.ones(tgt_len, tgt_len), diagonal=1).bool()
    decoder_mask = decoder_mask.unsqueeze(0).unsqueeze(0).to(device)
    return None, decoder_mask, None

# Improved training function with better error handling
def train_epoch_improved(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    num_batches = 0

    progress_bar = tqdm(dataloader, desc="Training")

    for batch_idx, (english_batch, sinhala_batch) in enumerate(progress_bar):
        try:
            optimizer.zero_grad()

            # Prepare target sequences (shift for teacher forcing)
            target_input = [START_TOKEN + sentence for sentence in sinhala_batch]
            target_output = [sentence + END_TOKEN for sentence in sinhala_batch]

            # Forward pass
            outputs = model(
                english_batch,
                target_input,
                encoder_self_attention_mask=None,
                decoder_self_attention_mask=None,
                decoder_cross_attention_mask=None,
                enc_start_token=False,
                enc_end_token=False,
                dec_start_token=False,
                dec_end_token=False
            )

            # Tokenize target output for loss calculation
            target_indices = []
            for sentence in target_output:
                indices = []
                for char in sentence:
                    if char in sinhala_to_index:
                        indices.append(sinhala_to_index[char])
                    else:
                        indices.append(sinhala_to_index[UNK_TOKEN])

                # Pad or truncate to max_sequence_length
                if len(indices) > max_sequence_length:
                    indices = indices[:max_sequence_length]
                else:
                    while len(indices) < max_sequence_length:
                        indices.append(sinhala_to_index[PADDING_TOKEN])
                target_indices.append(indices)

            target_tensor = torch.tensor(target_indices, dtype=torch.long).to(device)

            # Calculate loss
            loss = criterion(outputs.reshape(-1, outputs.size(-1)), target_tensor.reshape(-1))

            # Backward pass
            loss.backward()

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            total_loss += loss.item()
            num_batches += 1

            # Update progress bar
            progress_bar.set_postfix({'loss': loss.item()})

        except Exception as e:
            print(f"Error in batch {batch_idx}: {e}")
            print(f"English batch: {english_batch}")
            print(f"Sinhala batch: {sinhala_batch}")
            raise e

    return total_loss / num_batches


# Validation function
def validate_epoch(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    num_batches = 0

    with torch.no_grad():
        for batch_idx, (english_batch, sinhala_batch) in enumerate(dataloader):
            # Prepare target sequences
            target_input = [START_TOKEN + sentence for sentence in sinhala_batch]
            target_output = [sentence + END_TOKEN for sentence in sinhala_batch]

            # Forward pass
            outputs = model(
                english_batch,
                target_input,
                encoder_self_attention_mask=None,
                decoder_self_attention_mask=None,
                decoder_cross_attention_mask=None,
                enc_start_token=False,
                enc_end_token=False,
                dec_start_token=False,
                dec_end_token=False
            )

            # Tokenize target output for loss calculation
            target_indices = []
            for sentence in target_output:
                indices = [sinhala_to_index.get(char, sinhala_to_index['<UNK>']) for char in sentence]
                # Pad or truncate to max_sequence_length
                if len(indices) > max_sequence_length:
                    indices = indices[:max_sequence_length]
                else:
                    while len(indices) < max_sequence_length:
                        indices.append(sinhala_to_index[PADDING_TOKEN])
                target_indices.append(indices)

            target_tensor = torch.tensor(target_indices, dtype=torch.long).to(device)

            # Calculate loss
            loss = criterion(outputs.reshape(-1, outputs.size(-1)), target_tensor.reshape(-1))

            total_loss += loss.item()
            num_batches += 1

    return total_loss / num_batches

##Training

In [7]:
# Start training
print("Starting training...")
train_losses = []
val_losses = []
best_val_loss = float('inf')

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")

    # Training
    train_loss = train_epoch_improved(model, train_dataloader, optimizer, criterion, device)
    train_losses.append(train_loss)

    # Validation (similar improvements)
    model.eval()
    val_loss = 0
    val_batches = 0

    with torch.no_grad():
        for english_batch, sinhala_batch in val_dataloader:
            target_input = [START_TOKEN + sentence for sentence in sinhala_batch]
            target_output = [sentence + END_TOKEN for sentence in sinhala_batch]

            outputs = model(
                english_batch,
                target_input,
                encoder_self_attention_mask=None,
                decoder_self_attention_mask=None,
                decoder_cross_attention_mask=None,
                enc_start_token=False,
                enc_end_token=False,
                dec_start_token=False,
                dec_end_token=False
            )

            target_indices = []
            for sentence in target_output:
                indices = []
                for char in sentence:
                    if char in sinhala_to_index:
                        indices.append(sinhala_to_index[char])
                    else:
                        indices.append(sinhala_to_index[UNK_TOKEN])

                if len(indices) > max_sequence_length:
                    indices = indices[:max_sequence_length]
                else:
                    while len(indices) < max_sequence_length:
                        indices.append(sinhala_to_index[PADDING_TOKEN])
                target_indices.append(indices)

            target_tensor = torch.tensor(target_indices, dtype=torch.long).to(device)
            loss = criterion(outputs.reshape(-1, outputs.size(-1)), target_tensor.reshape(-1))

            val_loss += loss.item()
            val_batches += 1

    val_loss = val_loss / val_batches
    val_losses.append(val_loss)

    # Learning rate scheduling
    scheduler.step(val_loss)

    print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'epoch': epoch,
            'val_loss': val_loss,
            'train_loss': train_loss,
            'english_to_index': english_to_index,
            'sinhala_to_index': sinhala_to_index,
            'index_to_english': index_to_english,
            'index_to_sinhala': index_to_sinhala,
            'english_vocab': english_vocab,
            'sinhala_vocab': sinhala_vocab
        }, 'best_transformer_model.pth')
        print(f"New best model saved! Val Loss: {val_loss:.4f}")

    # Early stopping
    if epoch > 20 and val_loss > best_val_loss * 1.1:
        print("Early stopping triggered!")
        break

print(f"Training completed! Best validation loss: {best_val_loss:.4f}")

# Save vocabularies separately
with open('vocabularies.pkl', 'wb') as f:
    pickle.dump({
        'english_to_index': english_to_index,
        'index_to_english': index_to_english,
        'english_vocab': english_vocab,
        'sinhala_to_index': sinhala_to_index,
        'index_to_sinhala': index_to_sinhala,
        'sinhala_vocab': sinhala_vocab
    }, f)

print("Training completed successfully!")

Starting training...

Epoch 1/100


Training:   0%|          | 0/4 [00:00<?, ?it/s]

Error in batch 0: '<'
English batch: ('How are you?', 'See you later', 'I am sad', 'They are students', 'It is raining', 'The car is red', 'Have a nice day', 'We are friends')
Sinhala batch: ('ඔබ කෙසේද?', 'ඉදිරියෙදී හමුවෙමු', 'මට දුකයි', 'ඔවුන් සිසුන්', 'වැස්ස එනවා', 'කාර් එක රතුයි', 'හොඳ දවසක් ගත කරන්න', 'අපි යාළුවෝයි')


KeyError: '<'

In [ ]:
# Translation function with beam search
def translate_beam_search(model, english_sentence, english_to_index, sinhala_to_index, index_to_sinhala, beam_size=3, max_length=128):
    model.eval()

    with torch.no_grad():
        # Initialize beam with START token
        beams = [(START_TOKEN, 0.0)]  # (sequence, score)

        for _ in range(max_length):
            new_beams = []

            for sequence, score in beams:
                # Check if sequence ended
                if sequence.endswith(END_TOKEN):
                    new_beams.append((sequence, score))
                    continue

                # Forward pass
                outputs = model(
                    [english_sentence],
                    [sequence],
                    encoder_self_attention_mask=None,
                    decoder_self_attention_mask=None,
                    decoder_cross_attention_mask=None,
                    enc_start_token=False,
                    enc_end_token=False,
                    dec_start_token=False,
                    dec_end_token=False
                )

                # Get probabilities for next character
                next_char_logits = outputs[0, len(sequence), :]
                next_char_probs = torch.softmax(next_char_logits, dim=-1)

                # Get top k candidates
                top_k_probs, top_k_indices = torch.topk(next_char_probs, beam_size)

                # Create new beam candidates
                for prob, idx in zip(top_k_probs, top_k_indices):
                    next_char = index_to_sinhala[idx.item()]
                    new_sequence = sequence + next_char
                    new_score = score + torch.log(prob).item()
                    new_beams.append((new_sequence, new_score))

            # Select top beam_size beams
            beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[:beam_size]

            # Check if all beams ended
            if all(seq.endswith(END_TOKEN) for seq, _ in beams):
                break

        # Return best sequence
        best_sequence = beams[0][0]
        # Remove special tokens
        best_sequence = best_sequence.replace(START_TOKEN, '').replace(END_TOKEN, '')
        return best_sequence

# Simple greedy translation function
def translate_greedy(model, english_sentence, english_to_index, sinhala_to_index, index_to_sinhala, max_length=128):
    model.eval()
    with torch.no_grad():
        # Start with START token
        translated = START_TOKEN

        for _ in range(max_length):
            # Forward pass
            outputs = model(
                [english_sentence],
                [translated],
                encoder_self_attention_mask=None,
                decoder_self_attention_mask=None,
                decoder_cross_attention_mask=None,
                enc_start_token=False,
                enc_end_token=False,
                dec_start_token=False,
                dec_end_token=False
            )

            # Get the next character prediction
            next_char_logits = outputs[0, len(translated), :]
            next_char_id = torch.argmax(next_char_logits).item()
            next_char = index_to_sinhala[next_char_id]

            # Stop if END token is generated
            if next_char == END_TOKEN:
                break

            # Add to translated sentence
            translated += next_char

        # Remove START token
        translated = translated.replace(START_TOKEN, '')
        return translated

# Load the best model
print("Loading best model...")
checkpoint = torch.load('best_transformer_model.pth', map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
print(f"Loaded model from epoch {checkpoint['epoch']} with validation loss {checkpoint['val_loss']:.4f}")

##Testing

In [ ]:
# Test translation function
print("\n" + "="*50)
print("TESTING TRANSLATION")
print("="*50)

# Test with training samples
test_indices = random.sample(range(len(val_english)), min(5, len(val_english)))

for idx in test_indices:
    english_text = val_english[idx]
    expected_sinhala = val_sinhala[idx]

    translation_greedy = translate_greedy(model, english_text, english_to_index, sinhala_to_index, index_to_sinhala)

    print(f"English: {english_text}")
    print(f"Expected: {expected_sinhala}")
    print(f"Generated: {translation_greedy}")
    print("-" * 50)

In [ ]:
# Interactive translation
print("\n" + "="*50)
print("INTERACTIVE TRANSLATION")
print("="*50)
print("Enter English text to translate (type 'quit' to exit):")

while True:
    user_input = input("\nEnglish: ").strip()
    if user_input.lower() == 'quit':
        break

    if not user_input:
        continue

    try:
        # Try both translation methods
        translation_greedy = translate_greedy(model, user_input, english_to_index, sinhala_to_index, index_to_sinhala)
        print(f"Sinhala (Greedy): {translation_greedy}")

        # Uncomment the line below for beam search (slower but potentially better)
        # translation_beam = translate_beam_search(model, user_input, english_to_index, sinhala_to_index, index_to_sinhala)
        # print(f"Sinhala (Beam): {translation_beam}")

    except Exception as e:
        print(f"Error: {e}")
        print("This might happen if the input contains characters not in the vocabulary.")

print("Translation session ended!")

In [ ]:
# Save final model info
print("\n" + "="*50)
print("TRAINING SUMMARY")
print("="*50)
print(f"Total epochs: {num_epochs}")
print(f"Best validation loss: {best_val_loss:.4f}")
print(f"Final training loss: {train_losses[-1]:.4f}")
print(f"Final validation loss: {val_losses[-1]:.4f}")
print(f"English vocabulary size: {len(english_vocab)}")
print(f"Sinhala vocabulary size: {len(sinhala_vocab)}")
print(f"Model parameters: {total_params:,}")
print("="*50)